# Using Flow Forecast library
In this tutorial we use the flow forecast library to preform some basic traffic forecasting. Flow Forecast is a general purpose deep learning for times series forecasting package.

In [ ]:
!git clone http://github.com/AIStream-Peelout/flow-forecast #-b remove_versions # You can use a custom branch
import os 
os.chdir('flow-forecast')
!pip install --force-reinstall --upgrade -r requirements.txt
!python setup.py install develop
from flood_forecast.trainer import train_function

Cloning into 'flow-forecast'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 8163 (delta 171), reused 150 (delta 89), pack-reused 7913
Receiving objects: 100% (8163/8163), 3.31 MiB | 2.99 MiB/s, done.
Resolving deltas: 100% (5903/5903), done.
     |████████████████████████████████| 276kB 2.5MB/s 
     |████████████████████████████████| 6.8MB 8.3MB/s 
     |████████████████████████████████| 9.5MB 41.5MB/s 
     |████████████████████████████████| 748.8MB 21kB/s 
     |████████████████████████████████| 10.6MB 17.4MB/s 
     |████████████████████████████████| 286kB 45.4MB/s 
     |████████████████████████████████| 829kB 34.3MB/s 
     |████████████████████████████████| 2.9MB 37.0MB/s 
     |████████████████████████████████| 1.7MB 36.2MB/s 
     |████████████████████████████████| 92kB 8.0MB/s 
     |████████████████████████████████| 276kB 21.7MB/s 
     |██████████████████████████████

/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:454: UserWarning: Normalizing '0.091dev' to '0.91.dev0'
  warnings.warn(tmpl.format(**locals()))
running install
running bdist_egg
running egg_info
creating flood_forecast.egg-info
writing flood_forecast.egg-info/PKG-INFO
writing dependency_links to flood_forecast.egg-info/dependency_links.txt
writing requirements to flood_forecast.egg-info/requires.txt
writing top-level names to flood_forecast.egg-info/top_level.txt
writing manifest file 'flood_forecast.egg-info/SOURCES.txt'
package init file 'flood_forecast/__init__.py' not found (or not a regular file)
package init file 'flood_forecast/transformer_xl/__init__.py' not found (or not a regular file)
package init file 'flood_forecast/preprocessing/__init__.py' not found (or not a regular file)
package init file 'flood_forecast/da_rnn/__init__.py' not found (or not a regular file)
package init file 'flood_forecast/basic/__init__.py' not found (or not a regular file)
package init f

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


## Step One: Install and authenticate
In this first step we need to install the library and authenticate with Weights and Biases and GCP. 

In [ ]:
#!pip install --upgrade --force-reinstall wandb
!wandb login
# If you want to have your weights and JSON files stashed automatically uncomment
# os.environ["MODEL_BUCKET"] = "my-gcp-bucket-name"
# os.environ["ENVIRONMENT_GCP"] = "Colab"
# os.environ["GCP_PROJECT"] = "project_id"


wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
!wget -O train.csv https://raw.githubusercontent.com/xiaochus/TrafficFlowPrediction/master/data/train.csv 

--2020-10-16 17:36:22--  https://raw.githubusercontent.com/xiaochus/TrafficFlowPrediction/master/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 199681 (195K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 195.00K  --.-KB/s    in 0.05s   

2020-10-16 17:36:23 (3.77 MB/s) - ‘train.csv’ saved [199681/199681]



In [ ]:
# We will preform very basic data techniques to just get the weekday.
import pandas as pd
import datetime as datetime
df = pd.read_csv("train.csv")
df["day_of_week"] = df["5 Minutes"].map(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y %H:%M').weekday())
df["datetime"] = df['5 Minutes']

In [ ]:
df.to_csv('train.csv')
# Truly bizzare error? WTF?

## Step Two Define the Configuration File
Now that we have everything installed and our data properly working. We need to define a configuration file. The configuration files is composed of four major required sub-parts: model_params, dataset_params, inference_params,

In [ ]:
def make_config_file(file_path, train_end, valid_end):
  run = wandb.init(project="library_demos")
  wandb_config = wandb.config
  config_default={                 
    "model_name": "MultiAttnHeadSimple",
    "model_type": "PyTorch",
    "model_params": {
      "number_time_series":2,
      "seq_len":wandb_config["forecast_history"], 
      "output_seq_len":wandb_config["out_seq_length"],
      "forecast_length":wandb_config["out_seq_length"]
     },
    "dataset_params":
    {  "class": "default",
       "training_path": file_path,
       "validation_path": file_path,
       "test_path": file_path,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"],
       "forecast_length":wandb_config["out_seq_length"],
       "train_end": train_end,
       "valid_start":int(train_end+1),
       "valid_end": int(valid_end),
       "test_start":int(valid_end) + 1,
       "target_col": ["Lane 1 Flow (Veh/5 Minutes)"],
       "relevant_cols": ["Lane 1 Flow (Veh/5 Minutes)", "day_of_week"],
       "scaler": "StandardScaler", 
       "interpolate": False
    },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "Adam",
       "optim_params":
       {

       },
       "lr": wandb_config["lr"],
       "epochs": 10,
       "batch_size":wandb_config["batch_size"]
    },
    "GCS": False,
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE"],
   "inference_params":
   {     t
         "datetime_start":"2016-02-24",
          "hours_to_forecast":150, 
          "test_csv_path":file_path,
          "decoder_params":{
              "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1
          },
          "dataset_params":{
             "file_path": file_path,
             "forecast_history":wandb_config["forecast_history"],
             "forecast_length":wandb_config["out_seq_length"],
             "relevant_cols": ["Lane 1 Flow (Veh/5 Minutes)", "day_of_week"],
             "target_col": ["Lane 1 Flow (Veh/5 Minutes)"],
             "scaling": "StandardScaler",
             "interpolate_param": False
          }
      }
  }
  wandb.config.update(config_default)
  return config_default

So I'll briefly explain what is going on in this config file.  

## Step Three Define Wandb Sweep Config
Now that we have our global config we define the wandb one . In this file we include all the parameters we want to sweep over. 

In [ ]:
sweep_config = {
  "name": "Default sweep",
  "method": "random",
  "parameters": {
        "batch_size": {
            "values": [2, 3, 4]
        },
        "lr":{
            "values":[0.001, 0.01]
        },
        "forecast_history":{
            "values":[1, 2, 3, 5]
        },
        "out_seq_length":{
            "values":[1, 2, 3, 4]
        }
    }
}

## Step Four: Run Code and log results
Now that we have both config files it is time to train our model and log the results to Weights and Biases to analyze later. 

In [ ]:
from flood_forecast.trainer import train_function
import wandb
sweep_id = wandb.sweep(sweep_config)
os.environ["SWEEP_ID"] = sweep_id
#!wandb agent $SWEEP_ID
wandb.agent(sweep_id, lambda: train_function("PyTorch", make_config_file("train.csv", 4500, 6000)) )
#_secretagent(sweep_id, lambda: train_function("PyTorch", make_config_file("train.csv", 4500, 6000)))

Create sweep with ID: du7xfwi9
Sweep URL: https://wandb.ai/igodfried/uncategorized/sweeps/du7xfwi9


wandb: Agent Starting Run: yroy12mh with config:
wandb: 	batch_size: 3
wandb: 	forecast_history: 3
wandb: 	lr: 0.01
wandb: 	out_seq_length: 1
wandb: Currently logged in as: igodfried (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


interpolate should be below
Now loading and scaling train.csv
interpolate should be below
Now loading and scaling train.csv
interpolate should be below
Now loading and scaling train.csv
Using Wandb config:
{'batch_size': 3, 'forecast_history': 3, 'lr': 0.01, 'out_seq_length': 1, 'model_name': 'MultiAttnHeadSimple', 'model_type': 'PyTorch', 'model_params': {'number_time_series': 2, 'seq_len': 3, 'output_seq_len': 1, 'forecast_length': 1}, 'dataset_params': {'class': 'default', 'training_path': 'train.csv', 'validation_path': 'train.csv', 'test_path': 'train.csv', 'batch_size': 3, 'forecast_history': 3, 'forecast_length': 1, 'train_end': 4500, 'valid_start': 4501, 'valid_end': 6000, 'test_start': 6001, 'target_col': ['Lane 1 Flow (Veh/5 Minutes)'], 'relevant_cols': ['Lane 1 Flow (Veh/5 Minutes)', 'day_of_week'], 'scaler': 'StandardScaler', 'interpolate': False}, 'training_params': {'criterion': 'MSE', 'optimizer': 'Adam', 'optim_params': {}, 'lr': 0.01, 'epochs': 10, 'batch_size': 3}, 'G


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

plotting with CI now
